## Setup

Run the code below to load the scrambled message:


In [1]:
import pandas as pd

message = pd.read_csv("https://www.dropbox.com/s/lgpn3vmksk3ssdo/scrambled_message.txt?dl=1")['Word']

In [2]:
import re

In [3]:
message = message.to_frame()

In this activity, a "word" refers to any set of characters with no white space, even though they are not truly an English word.  That is, even though many of elements of the scrambled message vector are nonsense, and some have punctuation, you can consider each element to be a "word".

Beware!  The object named `message` is a **pandas Series** of strings.  If you want to use functions that expect a string, rather than a series, remember `.apply()` and `lambda` functions.




## Warm-up exercises

1. How many characters are in the scrambled message?
2. How many of these characters are white space?
3. How many words are in the scrambled message?
4. Show all the punctuation marks in the scrambled message.
5. Print out, in all capitals, the longest word in the scrambled message.
6. Print out every piece of a word that starts with the letter "m" and ends with the letter "z" in the scrambled message.

In [4]:
print(message.columns)


Index(['Word'], dtype='object')


In [5]:
#1

# Define the function to count characters based on a regex pattern
def count_chars(text, pattern=r"."):
    return len(re.findall(pattern, text))

# Apply the function to the DataFrame column 'Word' and create a new column 'char_count_all'
message['char_count_all'] = message['Word'].apply(lambda x: count_chars(x, r"."))

# 20 characters in each cell, so 20 * 127 = 2540 characters in total.


In [6]:
message['char_count_no_spaces'] = message['Word'].apply(lambda x: count_chars(x, r"\S"))


In [7]:
message["char_dif"] = message['char_count_all'] - message['char_count_no_spaces']
print(message['char_dif'].sum())

# 1652 out of 2540 characters are white space
# 127 words in the scrambled message.

1652


In [8]:
def find_punctuation(text):
    return re.findall(r"[^\w\s]", str(text))

message['punctuation'] = message['Word'].apply(lambda x: find_punctuation(x))

print(message['punctuation'])

0         [!]
1          []
2         [,]
3          []
4          []
        ...  
122        []
123       [.]
124       [?]
125    [., !]
126       [.]
Name: punctuation, Length: 127, dtype: object


In [9]:
longest_string = message.loc[message['Word'].str.len().idxmax(), 'Word']
print(longest_string.upper())

KAUDEVILLIANUGH?AOGHAJDBN


In [10]:
pattern = r'm.*?z'
mz = (message['Word'].str.findall(pattern))
matching_words = mz.explode().dropna().unique()
print(matching_words)

['mosz' 'maaz']




## Decode a message

Complete the following steps to decode the message.  

1. Remove any spaces before or after each word.
2. Any time you see the word "ugh", with any number of h's, followed by a punctuation mark, delete this.
3. No word should be longer than 16 characters. Drop all extra characters beyond 13 off the end of each word.
4. Replace all instances of exactly 2 a's with exactly 2 e's.
5. Replace all z's with t's.
6. Every word that ends in b, change that to a y.  *Hint: look out for punctuation!*
7. Every word that starts with k, change that to a v.  *Hint: look out for capitalization!*
8. Use `.join()` to recombine all your words into a message.
9. Find the movie this quote is from.

In [11]:
message["Word"] = message["Word"].str.strip()

In [12]:
message["Word"] = message["Word"].str.replace(r"ugh+h*[^\w\s]?", "", regex=True)

In [13]:
message["Word"] = message["Word"].str[: 13]

In [14]:
message["Word"] = message["Word"].str.replace(r"aa", "ee", regex=True)

In [15]:
message["Word"] = message["Word"].str.replace(r"z", "t", regex=True)

In [16]:
message["Word"] = message["Word"].str.replace(r'b[!.,?]*$', 'y', regex=True)

In [17]:
message["Word"] = message["Word"].str.replace(r'k', 'v', regex=True)

In [18]:
def replace_k_with_v(word):
    if re.match(r'^[Kk]', word):  # Check if the word starts with 'K' or 'k'
        # Replace with 'v' or 'V' based on original capitalization
        return 'V' + word[1:] if word[0].isupper() else 'v' + word[1:]
    return word
message["Word"] = message["Word"].apply(replace_k_with_v)

In [19]:
quote = ' '.join(message["Word"])
print(quote)

Voila! In view, a humble vaudevilliana veteran, cast vicariously as both victim and villain by the vicissitudes of fate. This visage, no mere veneer of vanity, is a vestige of the vox populi now vacant, vanished. However, this valorous visitation of a bygone vexation stands vivified, and has vowed to vanquish these venal and virulent vermin, van guarding vice and vouchsafing the violently vicious and voracious violation of volition. The only verdict is vengeance; a vendetta, held as a votive not in vain, for the value and veracity of such shall one day vindicate the vigilant and the virtuous. Verily this vichyssoise of verbiage veers most verbose, so let me simply add that its my very good honour to meet you and you may call me V.
